# Data

In [ ]:
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

# Ensure reproducibility
import random, numpy as np, torch
seed = 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# ----------------------------------------------------------------------
# Data Noise
data_noise = 0.05          # same as your 2-D example

# Alpha
alpha = 0.95
# Generating alphas to test
from utils_tools.utils_result_metrics import generating_alphas
alphas = generating_alphas(20)


# Define the 3-D Helmholtz PDE
from utils_pde.utils_pde_3dhelmholtz import Helmholtz3D
domain = ((0.0, 1.0), (0.0, 1.0), (0.0, 1.0))   # (x0,x1),(y0,y1),(z0,z1)
k = math.pi                                     # wave number

true_solution = (
    lambda xyz: torch.sin(math.pi * xyz[..., 0:1])
              * torch.sin(math.pi * xyz[..., 1:2])
              * torch.sin(math.pi * xyz[..., 2:3])
)

pde = Helmholtz3D(k=k, domain=domain, true_solution=true_solution)

# Generate training / testing / calibration data
(X_train, Y_train)         = pde.data_generation(1_000, data_noise)
(X_test,  Y_test)          = pde.data_generation(400, data_noise)
(X_calibration, Y_calibration) = pde.data_generation(200, data_noise)
(X_validation, Y_validation) = pde.data_generation(400, data_noise)
# Number of interior collocation points for the PINN residual
colloc_pt_num = 500

# --------------------------------------------
# Defining Plotting Grid
# --------------------------------------------

n_grid = 50
x = torch.linspace(domain[0][0], domain[0][1], n_grid)
y = torch.linspace(domain[1][0], domain[1][1], n_grid)
z = torch.linspace(domain[2][0], domain[2][1], n_grid)
X, Y, Z = torch.meshgrid(x, y, z, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1), Z.reshape(-1, 1)], dim=1)


Using device: cpu
Using device: cpu


/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Feature Distance Model

## Define Model

In [2]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = dict(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)

# CP-Model
cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"feature", # Change base on if the baseline cp
    "k":100
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"feature",
    "n_samples":10, 
}

dist_feat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [3]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_feat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_feat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=10, 
    heuristic_u="feature"
)
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=5.16e+01 | lr=1.0e-03
ep   500 | L=9.12e-01 | lr=1.0e-03
ep  1000 | L=4.78e-01 | lr=1.0e-03
ep  1500 | L=1.83e-01 | lr=1.0e-03
ep  2000 | L=1.16e-01 | lr=1.0e-03
ep  2500 | L=8.15e-02 | lr=1.0e-03
ep  3000 | L=8.34e-02 | lr=1.0e-03
ep  3500 | L=7.18e-02 | lr=1.0e-03
ep  4000 | L=9.54e-02 | lr=1.0e-03
ep  4500 | L=2.15e-02 | lr=1.0e-03
ep  5000 | L=1.74e-02 | lr=1.0e-03


In [4]:
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)

## Create CP Model

In [5]:
# CP Model
dist_feat_pinn_cp = CP(dist_feat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"feature",  # Change this based on cp
        "k":100
}

dist_feat_pinn_cp_cal_predset = dist_feat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [6]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_feat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_feat_pinn, **baseline_testing_args)
dist_feat_pinndf_cal = cp_test_uncertainties(dist_feat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical

100%|██████████| 19/19 [00:01<00:00, 14.00it/s]


In [ ]:
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="feature_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_feat_pinn_uncal_predset, dist_feat_pinn_cp_cal_predset, 
    true_solution, 
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal,
    title="3D UQ", main_title="Feature Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)



save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="feature_distance_coverage",
    loss=baseline_data_loss
)(
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal, 
    main_title="Feature Distance Model Metrics",
)

# Latent Distance Model

## Define Model

In [7]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = dict(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)

baseline_pred_kwargs = dict(n_samples=200)


# CP-Model

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"latent", # Change base on if the baseline cp
    "k":100
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"latent",
    "n_samples":10, 
}

dist_lat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [8]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_lat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_lat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=10, 
    heuristic_u="latent"
)
dist_lat_pinn_uncal_predset = dist_lat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=3.85e+01 | lr=1.0e-03
ep   500 | L=3.09e-01 | lr=1.0e-03
ep  1000 | L=6.22e-02 | lr=1.0e-03
ep  1500 | L=4.88e-02 | lr=1.0e-03
ep  2000 | L=1.68e-02 | lr=1.0e-03
ep  2500 | L=2.24e-02 | lr=1.0e-03
ep  3000 | L=1.41e-02 | lr=1.0e-03
ep  3500 | L=2.94e-02 | lr=1.0e-03
ep  4000 | L=4.81e-02 | lr=1.0e-03
ep  4500 | L=9.55e-03 | lr=1.0e-03
ep  5000 | L=1.43e-02 | lr=1.0e-03


## Create CP Model

In [9]:
# CP Model
dist_lat_pinn_cp = CP(dist_lat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"latent",  # Change this based on cp
        "k":100
}

dist_lat_pinn_cp_cal_predset = dist_lat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_lat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_lat_pinn, **baseline_testing_args)
dist_lat_pinn_df_cal = cp_test_uncertainties(dist_lat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="latent_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_lat_pinn_uncal_predset, dist_lat_pinn_cp_cal_predset, 
    true_solution, 
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    title="2D UQ", main_title="Latent Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)


save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="latent_distance_coverage",
    loss=baseline_data_loss
)(
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    main_title="Latent Distance Model Coverage Plots"
)

100%|██████████| 19/19 [00:00<00:00, 110.30it/s]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

# Dropout Model

## Define Model

In [11]:
from utils_uqmd.utils_uq_dropout import DropoutPINN

# Base Model Instance
model_args = dict(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)
baseline_pred_kwargs = {  
    "n_samples":20 
}

# CP Model
cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline cp
    "k":100
}

cp_pred_kwargs = {
    "X_train":X_train,  "Y_train":Y_train,
    "X_cal":X_calibration, "Y_cal":Y_calibration,
    "heuristic_u":"raw_std",
    "k":100
}

baseline_testing_args = { 
    # "uqmodel":do_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "n_samples":20
}

dropout_pinn = DropoutPINN(**model_args)

## Training Base Model

In [12]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_ic=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dropout_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dropout_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=20
)
dropout_pinn_uncal_predset = dropout_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     1 | L=3.61e+01 | data=2.42e-01 | pde=3.56e+01  ic=0.00e+00  bc=2.83e-02 | lr=1.00e-03
ep   500 | L=8.37e+00 | data=9.29e-02 | pde=7.51e+00  ic=0.00e+00  bc=7.66e-02 | lr=1.00e-03
ep  1000 | L=4.90e+00 | data=6.86e-02 | pde=4.21e+00  ic=0.00e+00  bc=6.16e-02 | lr=1.00e-03
ep  1500 | L=4.14e+00 | data=4.38e-02 | pde=3.57e+00  ic=0.00e+00  bc=5.20e-02 | lr=1.00e-03
ep  2000 | L=3.83e+00 | data=4.38e-02 | pde=3.28e+00  ic=0.00e+00  bc=5.04e-02 | lr=1.00e-03
ep  2500 | L=3.39e+00 | data=4.74e-02 | pde=2.84e+00  ic=0.00e+00  bc=5.01e-02 | lr=1.00e-03
ep  3000 | L=3.20e+00 | data=4.83e-02 | pde=2.57e+00  ic=0.00e+00  bc=5.79e-02 | lr=1.00e-03
ep  3500 | L=3.12e+00 | data=4.31e-02 | pde=2.60e+00  ic=0.00e+00  bc=4.81e-02 | lr=1.00e-03
ep  4000 | L=3.19e+00 | data=4.48e-02 | pde=2.65e+00  ic=0.00e+00  bc=4.91e-02 | lr=1.00e-03
ep  4500 | L=2.57e+00 | data=4.63e-02 | pde=2.03e+00  ic=0.00e+00  bc=4.98e-02 | lr=1.00e-03
ep  5000 | L=2.41e+00 | data=4.46e-02 | pde=1.89e+00  ic=0.00e+00  bc=

/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


## Create CP Model

In [13]:
# CP Model
dropout_pinn_cp = CP(dropout_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",  # Change this based on cp
        "k":100
}

dropout_pinn_cp_cal_predset = dropout_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dropout_pinn_df_uncal = do_test_uncertainties(uqmodel=dropout_pinn, **baseline_testing_args)
dropout_pinn_df_cal = cp_test_uncertainties(dropout_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="dropout",
    loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)


save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="dropout_coverage",
    loss=baseline_data_loss
)(
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    main_title="Dropout Model Coverage Plots"
)

100%|██████████| 19/19 [00:03<00:00,  5.69it/s]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

# VI Model

## Define Model

In [15]:
from utils_uqmd.utils_uq_vi import VIBPINN

# Base Model Instance
model_args = {
    "pde_class":pde,
    "input_dim":3,
    "hidden_dims":[32, 64, 128, 128, 128, 64, 32], 
    "output_dim":1,
    "mu_std" : 0.01, "rho" : -2, "prior_std" : 1.0, 
    "init_data_noise" : 1.0, "learn_data_noise" : False, 
    "act_func" : nn.Tanh()
}
baseline_pred_kwargs = {
    "n_samples":5000 
}

baseline_testing_args = { 
    # "uqmodel":vi_model, 
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test
}

vi_bpinn = VIBPINN(**model_args)

## Training Base Model

In [16]:
# Base Model
# Training
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}
fit_kwargs_grid = {
    "epochs":20000,
    "λ_pde":5.0, "λ_bc":5.0, "λ_elbo":1.0,  
    "lr":3e-3,
    "stop_schedule":20000
}
baseline_loss_dict = vi_bpinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = vi_bpinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=20
)
vi_bpinn_uncal_predset = vi_bpinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


ep     0 | L=1.06e+04 | elbo=1.65e+02 | pde=2.10e+03  ic=0.00e+00  bc=9.67e-01 | lr=3.00e-03 | learned noise_std=1.000e+00
ep     1 | L=1.85e+04 | elbo=1.97e+02 | pde=3.66e+03  ic=0.00e+00  bc=3.36e-03 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   200 | L=4.10e+03 | elbo=4.32e+02 | pde=7.33e+02  ic=0.00e+00  bc=8.63e-03 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   400 | L=2.45e+03 | elbo=2.25e+02 | pde=4.44e+02  ic=0.00e+00  bc=1.64e-02 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   600 | L=3.44e+04 | elbo=1.84e+02 | pde=6.85e+03  ic=0.00e+00  bc=8.47e-02 | lr=3.00e-03 | learned noise_std=1.000e+00
ep   800 | L=3.04e+03 | elbo=1.40e+02 | pde=5.80e+02  ic=0.00e+00  bc=2.12e-01 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  1000 | L=1.11e+04 | elbo=2.86e+02 | pde=2.17e+03  ic=0.00e+00  bc=1.87e-01 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  1200 | L=8.46e+03 | elbo=4.22e+02 | pde=1.61e+03  ic=0.00e+00  bc=3.07e-01 | lr=3.00e-03 | learned noise_std=1.000e+00
ep  1400 | L=5.7

## Create CP Model

In [17]:
# CP Model
vi_bpinn_cp = CP(vi_bpinn)

# CP
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",
        "k":10
}

cp_testing_args = {
        "alphas":alphas, 
        "X_test":X_test, "Y_test":Y_test, 
        "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
        "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
        "k":10
}

vi_bpinn_cp_cal_predset = vi_bpinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
vi_bpinn_df_uncal = vi_test_uncertainties(uqmodel=vi_bpinn, **baseline_testing_args)
vi_bpinn_df_cal = cp_test_uncertainties(vi_bpinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="vi_bpinn",
    loss=baseline_data_loss
)(
    grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
    true_solution, 
    vi_bpinn_df_uncal, vi_bpinn_df_cal,
    title="2D UQ", main_title="VI Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)


save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="vi_bpinn_coverage",
    loss=baseline_data_loss
)(
    vi_bpinn_df_uncal, vi_bpinn_df_cal,
    mian_title = "VI BPINN Coverage Plots"
)

100%|██████████| 19/19 [5:24:40<00:00, 1025.28s/it]  


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

# HMC Model

## Define Model

In [19]:
from utils_uqmd.utils_uq_hmc import HMCBPINN

# Base Model Instance
model_args = dict(
    pde_class=pde, input_dim=3, 
    hidden_dims=[32, 64, 128, 128, 128, 64, 32],
    output_dim=1, act_func=nn.Tanh, prior_std=1.0,
    step_size=1e-3, leapfrog_steps=5
)

baseline_testing_args = {
    # "uqmodel": hmc_model,
    "alphas": alphas,
    "X_test": X_test,
    "Y_test": Y_test,
    "n_samples": 5000
}

baseline_pred_kwargs = {"n_samples": 5000}

hmc_bpinn = HMCBPINN(**model_args)

## Training Base Model

In [20]:
# Base Model
# Training
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}
fit_kwargs_grid = {
    "λ_pde": 3.0,
    "λ_bc": 5.0,
    "λ_data": 1.0,
    "epochs": 500,
    "lr":1e-3,
    "hmc_samples": 12000,
    "brun_in":5000,
    "step_size": 1e-4,
    "leapfrog_steps": 13
}
baseline_loss_dict = hmc_bpinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = hmc_bpinn.data_loss(X_validation, Y_validation)

hmc_bpinn_uncal_predset = hmc_bpinn.predict(
            alpha=alpha, X_test=X_test,
            **baseline_pred_kwargs
        )

[MAP] epoch    500 −logPost=2.195e+01  Data=6.35e-01  PDE=2.03e+01  IC=0.00e+00  BC=2.50e-01


HMC:   4%|▍         | 500/12000 [2:23:07<1:12:34,  2.64it/s, acc=0.64]    

[HMC] iter    500  acc-rate=0.64


HMC:   8%|▊         | 1000/12000 [2:26:21<1:22:01,  2.24it/s, acc=0.64]

[HMC] iter   1000  acc-rate=0.64


...

HMC:  92%|█████████▏| 11000/12000 [3:43:04<07:22,  2.26it/s, acc=0.65]

[HMC] iter  11000  acc-rate=0.65


HMC:  96%|█████████▌| 11500/12000 [3:46:35<03:29,  2.38it/s, acc=0.64]

[HMC] iter  11500  acc-rate=0.64


[HMC] iter  12000  acc-rate=0.64
Finished HMC: avg acceptance 0.645
Kept 11500 posterior samples


## Create CP Model

In [21]:
# CP Model
hmc_bpinn_cp = CP(hmc_bpinn)

# CP
cp_pred_kwargs = { 
    "X_train": X_train, "Y_train": Y_train,
    "X_cal": X_calibration, "Y_cal": Y_calibration,
    "heuristic_u": "raw_std", "k": 100
}

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
    "k":100
}

hmc_bpinn_cp_cal_predset = hmc_bpinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import hmc_test_uncertainties, cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
hmc_bpinn_df_uncal = hmc_test_uncertainties(uqmodel=hmc_bpinn, **baseline_testing_args)
hmc_bpinn_df_cal = cp_test_uncertainties(hmc_bpinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="hmc_bpinn",
    loss=baseline_data_loss
)(
    grid_test, hmc_bpinn_uncal_predset, hmc_bpinn_cp_cal_predset, 
    true_solution, 
    hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
    title="2D UQ", main_title="HMC Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)

save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="hmc_bpinn_coverage",
    loss=baseline_data_loss
)(
    hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
    main_title="HMC BPINN Coverage Plots"
)

  0%|          | 0/19 [00:00<?, ?it/s]/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
100%|██████████| 19/19 [01:55<00:00,  6.08s/it]


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>